# Thực hành tuần 2 - Introduction to Data Science

**Preprocess the data**

1 - Ingestion data


2 - Find the problems

+ Missing values
+ duplicated
+ Bất đồng bộ về kiểu dữ liệu
+ Outliers ()
+ Logic

3 - Clean


-> EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Khởi tạo dữ liệu
data = pd.DataFrame(
    {'date':['01/01/2022','02/01/2022','03/01/2022','04/01/2022','05/01/2022','06/01/2022','06/01/2022'],
     'volumes_max':[120,123,102,None,111,None,124],
     'volumes':[120,130,102,None,111,None,124],
     'temperature':[22,23,21,36,None,31,25]
    }
)

In [3]:
data

,date,volumes_max,volumes,temperature
0,01/01/2022,120.0,120.0,22.0
1,02/01/2022,123.0,130.0,23.0
2,03/01/2022,102.0,102.0,21.0
3,04/01/2022,NaN,NaN,36.0
4,05/01/2022,111.0,111.0,NaN
5,06/01/2022,NaN,NaN,31.0
6,06/01/2022,124.0,124.0,25.0


## 1. Kiểm tra vấn đề dữ liệu

Một số vấn đề dữ liệu ta sẽ thực hiện kiểm tra:

- Missing value
- Duplicated
- Lỗi logic
- Bất đồng bộ

### 1.1 Kiểm tra dữ liệu bị thiếu

#### a. Kiểm tra trên 1 cột

In [4]:
data['date'].isnull()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
Name: date, dtype: bool

#### b. Kiểm tra trên bảng

In [5]:
data.isnull()

,date,volumes_max,volumes,temperature
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,True,True,False
4,False,False,False,True
5,False,True,True,False
6,False,False,False,False


**Đếm số lượng missing value trong 1 cột**

In [6]:
data.isnull().sum()

date           0
volumes_max    2
volumes        2
temperature    1
dtype: int64

**Lưu ý:**
- Liệu thực sự cột đó bị missing value?
- Phương pháp xử lý dữ liệu đối với cột đó như nào?

-> volume_max số lượng nước mưa tối đa. Liệu hôm đó không mưa sẽ được đại diện bằng giá trị nào

-> Đưa ra phương pháp với cột temperature

### 1.2 Kiểm tra dữ liệu bị lặp

[**.duplicated( subset, keep )**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html)

- **subset:** Danh sách cột để so sánh
- **keep:** 

    + 'first'(Mặc định): Đánh dấu tất cả bằng True, trừ lần xuất hiện đầu tiên 
    + 'last': Tương tự như first, trừ lần xuất hiện cuối cùng
    + False: Tất cả là True

#### a. Kiểm tra xem có dòng nào giống nhau


In [7]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool

#### b. Kiểm tra xem tại một cột có dòng nào giống nhau


In [8]:
data.duplicated(subset = ['date'])

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [9]:
data[data.duplicated(subset = ['date'])]

,date,volumes_max,volumes,temperature
6,06/01/2022,124.0,124.0,25.0


Nhận xét: 
- Dữ liệu khảo sát theo cột ngày bị lặp lại tại ngày 06/01/2022

## 2. Làm sạch dữ liệu

### 2.1 Dữ liệu bị thiếu



#### a. Thay thế bằng một giá trị cụ thể
Hàm hỗ trợ gán giá trị cho missing value

[**.fillna(value, method, inplace):**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)

- **value:** Giá trị được thay thế
- **method:**
    
    + ‘backfill’/‘bfill’: Thay thế bằng giá trị đằng sau
    + ‘pad’ / ‘ffill’: Thay thế bằng giá trị đằng trước
    + None(Mặc định)
    
- **inplace:** 
    + False(Mặc định): Chỉ trả về DataFrame hoặc Series đã được thay đổi
    + True: Thay đổi trực tiếp trên DataFrame và Series hiện tại

**Thay thế bằng giá trị 0 cho một cột**

In [10]:
data['volumes'].fillna(0)

0    120.0
1    130.0
2    102.0
3      0.0
4    111.0
5      0.0
6    124.0
Name: volumes, dtype: float64

In [11]:
data['volumes']

0    120.0
1    130.0
2    102.0
3      NaN
4    111.0
5      NaN
6    124.0
Name: volumes, dtype: float64

In [12]:
data['volumes'].fillna(0,inplace = True)

In [13]:
data['volumes']

0    120.0
1    130.0
2    102.0
3      0.0
4    111.0
5      0.0
6    124.0
Name: volumes, dtype: float64

**Thay thế bằng giá trị đứng trước**

In [14]:
data['volumes_max']

0    120.0
1    123.0
2    102.0
3      NaN
4    111.0
5      NaN
6    124.0
Name: volumes_max, dtype: float64

In [15]:
data['volumes_max'].fillna(method='ffill')

0    120.0
1    123.0
2    102.0
3    102.0
4    111.0
5    111.0
6    124.0
Name: volumes_max, dtype: float64

In [16]:
data['volumes_max'].fillna(data['volumes_max'].mean())

0    120.0
1    123.0
2    102.0
3    116.0
4    111.0
5    116.0
6    124.0
Name: volumes_max, dtype: float64

#### b. Loại bỏ dữ liệu bị thiếu
[**.dropna(axis, how, subset, inplace)**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html/)

- **axis:**
    
    + 0 hoăc index(Mặc định): Xóa các dòng missing value
    + 1 hoặc columns: Xóa các cột missing value
- **how:**

    + 'any'(Mặc định): Nếu xuất hiện giá trị là N/A xóa bỏ dòng hoặc cột đó (Phụ thuộc vào axis)
    + 'all': Nếu tất cả giá trị là N/A, xóa bỏ dòng hoặc cột đó.
    
- **inplace:** Tương tự fillna()


In [17]:
data.dropna() #axis =0

,date,volumes_max,volumes,temperature
0,01/01/2022,120.0,120.0,22.0
1,02/01/2022,123.0,130.0,23.0
2,03/01/2022,102.0,102.0,21.0
6,06/01/2022,124.0,124.0,25.0


In [18]:
data.dropna(axis = 1)

,date,volumes
0,01/01/2022,120.0
1,02/01/2022,130.0
2,03/01/2022,102.0
3,04/01/2022,0.0
4,05/01/2022,111.0
5,06/01/2022,0.0
6,06/01/2022,124.0


### 2.2 Dữ liệu bị lặp lại

#### Loại bỏ dữ liệu

[**.drop_duplicates(subset, keep, inplace, ignore_index)**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

- **subset:** Tập hợp danh sách các cột được xét làm định danh cho dữ liệu
- **keep:** tương tự như duplicated
- **inplace:** tương tự như trên
- **ignore_index:** 
    
    + False(Mặc định):  Sẽ không đánh lại index
    + True: Ngược lại với False, các dòng sẽ được đánh lại bắt đầu từ 0


In [19]:
data.drop_duplicates(subset=['date'],keep='last',ignore_index=False)

,date,volumes_max,volumes,temperature
0,01/01/2022,120.0,120.0,22.0
1,02/01/2022,123.0,130.0,23.0
2,03/01/2022,102.0,102.0,21.0
3,04/01/2022,NaN,0.0,36.0
4,05/01/2022,111.0,111.0,NaN
6,06/01/2022,124.0,124.0,25.0


### 2.3 Tích hợp các bảng dữ liệu

**Bảng dữ liệu tương ứng**

In [20]:
data1 = pd.DataFrame(
    {'date':['01/01/2022','02/01/2022','03/01/2022','04/01/2022','05/01/2022','06/01/2022'],
     'air':[22,23,21,36,31,25]
    }
)

In [21]:
data2 = data.drop_duplicates(subset=['date']) # dữ liệu sau khi làm sạch

In [22]:
data1

,date,air
0,01/01/2022,22
1,02/01/2022,23
2,03/01/2022,21
3,04/01/2022,36
4,05/01/2022,31
5,06/01/2022,25


In [23]:
data2

,date,volumes_max,volumes,temperature
0,01/01/2022,120.0,120.0,22.0
1,02/01/2022,123.0,130.0,23.0
2,03/01/2022,102.0,102.0,21.0
3,04/01/2022,NaN,0.0,36.0
4,05/01/2022,111.0,111.0,NaN
5,06/01/2022,NaN,0.0,31.0


#### a. Kết hợp 2 bảng bằng hàm có sẵn



**Cách 1:**

[**pd.merge(right, left, on)**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

- **right:** Bảng thứ nhất
- **left:** Bảng thứ hai
- **on(Mặc định là None):** Trong trường hợp là None, kết hợp dựa trên index của hai bảng với nhau. Nếu lựa chọn cột làm điểm chung sẽ kết hợp hai bảng theo cột đó 

In [24]:
pd.merge(right = data2,left = data1,on='date')

,date,air,volumes_max,volumes,temperature
0,01/01/2022,22,120.0,120.0,22.0
1,02/01/2022,23,123.0,130.0,23.0
2,03/01/2022,21,102.0,102.0,21.0
3,04/01/2022,36,NaN,0.0,36.0
4,05/01/2022,31,111.0,111.0,NaN
5,06/01/2022,25,NaN,0.0,31.0


**Cách 1:**

[**.join(other, on)**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)

- **other:** Là bảng dữ liệu kết hợp với bảng hiện tại
- **on(Mặc định là None):** Mô tả tên cột tại bảng hiện tại được kết hợp làm chỉ mục với index của bảng bổ sung

In [25]:
data2.join(data1.set_index('date'),on='date')

,date,volumes_max,volumes,temperature,air
0,01/01/2022,120.0,120.0,22.0,22
1,02/01/2022,123.0,130.0,23.0,23
2,03/01/2022,102.0,102.0,21.0,21
3,04/01/2022,NaN,0.0,36.0,36
4,05/01/2022,111.0,111.0,NaN,31
5,06/01/2022,NaN,0.0,31.0,25


#### b. Truy vấn SQL giữa các bảng

In [26]:
# Thực hành

## 3. Ví dụ khác

In [27]:
data = pd.DataFrame(
    {
        'MSV':['A43471','40401','A39380','A38843','A41131','A30426','A30479','A33541','A30012','A31111','A42356','A43321','A35614','40401'],
         'DiemQT':[1,7,6,7,5,10,6,7,9,5,1,None,8,6],
        'GioiTinh':['Nam','Nam','Nu','Nam','Nu','Nam','Nam','Nam',0,1,1,0,1,0]
    }
)

In [28]:
data

,MSV,DiemQT,GioiTinh
0,A43471,1.0,Nam
1,40401,7.0,Nam
2,A39380,6.0,Nu
3,A38843,7.0,Nam
4,A41131,5.0,Nu
5,A30426,10.0,Nam
6,A30479,6.0,Nam
7,A33541,7.0,Nam
8,A30012,9.0,0
9,A31111,5.0,1


In [29]:
data.dtypes

MSV          object
DiemQT      float64
GioiTinh     object
dtype: object

### 3.1 Xử lý bất đồng bộ

In [30]:
data['GioiTinh'].value_counts()

Nam    6
0      3
1      3
Nu     2
Name: GioiTinh, dtype: int64

In [31]:
data[data['GioiTinh']==0]['GioiTinh']

8     0
11    0
13    0
Name: GioiTinh, dtype: object

In [32]:
data['GioiTinh']= data['GioiTinh'].replace(0,'Nu')

In [33]:
data

,MSV,DiemQT,GioiTinh
0,A43471,1.0,Nam
1,40401,7.0,Nam
2,A39380,6.0,Nu
3,A38843,7.0,Nam
4,A41131,5.0,Nu
5,A30426,10.0,Nam
6,A30479,6.0,Nam
7,A33541,7.0,Nam
8,A30012,9.0,Nu
9,A31111,5.0,1
